# Generative AI Day 5

## Now we will take our project from Day 1 to next level

### **Business Challenge:**

Create a product that builds a travel Brochure from a tourism website of a country for travellers

We will provide a country name and their primary website

In [1]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [3]:
# Initialize constants

load_dotenv(override=True)
api_key=os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key) > 10:
    print("API key looks good so far")
else:
    print("there might be some problem with your API key. Please check")
MODEL = "gpt-4o-mini"
openai = OpenAI()

API key looks good so far


In [16]:
# A class to represent a webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """
    def __init__(self, url):
        self.url = url
        response = requests.get(self.url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "img", "style", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator='\n', strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if links]

    def get_contents(self):
        return f"Webpage Title: \n{self.title}\nWebpage contents: \n{self.text}\n\n"

In [26]:
esp = Website("https://www.spain.info/en/")
esp.links

['/en/',
 '#',
 '/en/destination/madrid/',
 '/en/destination/barcelona/',
 '/en/destination/valencia/',
 '/en/destination/seville/',
 '/en/destination/zaragoza/',
 '/en/destination/malaga/',
 '/en/destination/bilbao/',
 '/en/destination/santiago-compostela/',
 '/en/destination/salamanca/',
 '/en/destination/santander/',
 '/en/destination/caceres/',
 '/en/region/canary-islands/',
 '/en/region/balearic-islands/',
 '/en/destinations/',
 '/en/destinations/',
 '#',
 '/en/art-culture/',
 '/en/coasts-beaches/',
 '/en/cities/',
 '/en/nature/',
 '/en/gastronomy-wine-tourism/',
 '/en/adventure-sports/',
 '/en/shopping/',
 '/en/routes/',
 '#',
 'https://sustainability.spain.info/en/',
 '/en/about-spain/',
 '/en/info/',
 '/en/how-to-get-there/',
 '/en/getting-around/',
 '/en/activities/',
 '/en/route-planner/',
 '/en/brochures/',
 '/en/weather/',
 '/en/travel-tips/visa-passport/',
 'https://sustainability.spain.info/en/',
 '/en/about-spain/',
 '/en/info/',
 '/en/how-to-get-there/',
 '/en/getting-a

### First step: Have gpt-4o-mini figure out which links are relevant 

#### Use gpt-4o-mini to read the links on a webpage, and respond in structured JSON.

It should decide which links are relevant, and replace relative links such as `/en/destination/valencia/` with `https://www.spain.info/en/destination/valencia/`.
We will use "one shot prompting" in which we provide an example of how to it should respond in a prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding, Imagine trying to code without LLM by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [18]:
link_system_prompt = "You are provided with the list of links found on a official tourism website. \
You are able to decide which of the links would be most relevant to include in a brochure about the \
travel guide for the destination in the given website such as links\
shopping, adventure sports, routes, culture or cities pages.\n"
link_system_prompt += "You should respond in a JSON as in this example:"
link_system_prompt += """
{
    "links":[
        {"type": "shopping page", "url": "https://full.url/goes/here/about"}
        {"type": "adventure-sports page", "url": "https://full.url/adventure-sports"}
    ]
}
"""

In [20]:
print(link_system_prompt)

You are provided with the list of links found on a official tourism website. You are able to decide which of the links would be most relevant to include in a brochure about the travel guide for the destination in the given website such as linksshopping, adventure sports, routes, culture or cities pages.
You should respond in a JSON as in this example:
{
    "links":[
        {"type": "shopping page", "url": "https://full.url/goes/here/about"}
        {"type": "adventure-sports page", "url": "https://full.url/adventure-sports"}
    ]
}



In [21]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a travel brochure about the destination, respond with the full https URL in JSON format. \
    Do not include Terms ofservice, Privacy, email links. \n"
    user_prompt += "Links (some might be irrelevant links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [27]:
print(get_links_user_prompt(esp))

Here is the list of links on the website of https://www.spain.info/en/ - please decide which of these are relevant web links for a travel brochure about the destination, respond with the full https URL in JSON format.     Do not include Terms ofservice, Privacy, email links. 
Links (some might be irrelevant links):
/en/
#
/en/destination/madrid/
/en/destination/barcelona/
/en/destination/valencia/
/en/destination/seville/
/en/destination/zaragoza/
/en/destination/malaga/
/en/destination/bilbao/
/en/destination/santiago-compostela/
/en/destination/salamanca/
/en/destination/santander/
/en/destination/caceres/
/en/region/canary-islands/
/en/region/balearic-islands/
/en/destinations/
/en/destinations/
#
/en/art-culture/
/en/coasts-beaches/
/en/cities/
/en/nature/
/en/gastronomy-wine-tourism/
/en/adventure-sports/
/en/shopping/
/en/routes/
#
https://sustainability.spain.info/en/
/en/about-spain/
/en/info/
/en/how-to-get-there/
/en/getting-around/
/en/activities/
/en/route-planner/
/en/broc

In [23]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [28]:
esp = Website("https://www.spain.info/en/")
esp.links

['/en/',
 '#',
 '/en/destination/madrid/',
 '/en/destination/barcelona/',
 '/en/destination/valencia/',
 '/en/destination/seville/',
 '/en/destination/zaragoza/',
 '/en/destination/malaga/',
 '/en/destination/bilbao/',
 '/en/destination/santiago-compostela/',
 '/en/destination/salamanca/',
 '/en/destination/santander/',
 '/en/destination/caceres/',
 '/en/region/canary-islands/',
 '/en/region/balearic-islands/',
 '/en/destinations/',
 '/en/destinations/',
 '#',
 '/en/art-culture/',
 '/en/coasts-beaches/',
 '/en/cities/',
 '/en/nature/',
 '/en/gastronomy-wine-tourism/',
 '/en/adventure-sports/',
 '/en/shopping/',
 '/en/routes/',
 '#',
 'https://sustainability.spain.info/en/',
 '/en/about-spain/',
 '/en/info/',
 '/en/how-to-get-there/',
 '/en/getting-around/',
 '/en/activities/',
 '/en/route-planner/',
 '/en/brochures/',
 '/en/weather/',
 '/en/travel-tips/visa-passport/',
 'https://sustainability.spain.info/en/',
 '/en/about-spain/',
 '/en/info/',
 '/en/how-to-get-there/',
 '/en/getting-a

In [29]:
get_links("https://www.spain.info/en/")

{'links': [{'type': 'cities page',
   'url': 'https://www.spain.info/en/destination/madrid/'},
  {'type': 'cities page',
   'url': 'https://www.spain.info/en/destination/barcelona/'},
  {'type': 'cities page',
   'url': 'https://www.spain.info/en/destination/valencia/'},
  {'type': 'cities page',
   'url': 'https://www.spain.info/en/destination/seville/'},
  {'type': 'cities page',
   'url': 'https://www.spain.info/en/destination/zaragoza/'},
  {'type': 'cities page',
   'url': 'https://www.spain.info/en/destination/malaga/'},
  {'type': 'cities page',
   'url': 'https://www.spain.info/en/destination/bilbao/'},
  {'type': 'cities page',
   'url': 'https://www.spain.info/en/destination/santiago-compostela/'},
  {'type': 'cities page',
   'url': 'https://www.spain.info/en/destination/salamanca/'},
  {'type': 'cities page',
   'url': 'https://www.spain.info/en/destination/santander/'},
  {'type': 'cities page',
   'url': 'https://www.spain.info/en/destination/caceres/'},
  {'type': 'regio

### Second step: Make a brochure!

Assemble all the details into another prompt to GPT4-o

In [48]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    #print("Found links: ", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link['url']).get_contents()
    return result

In [49]:
print(get_all_details("https://www.spain.info/en/"))

Landing page:
Webpage Title: 
Spanish Tourism | Tourist information on Spain | spain.info
Webpage contents: 
Spain's official tourism website
Search
Where to go
Discover
our destinations
Madrid
Barcelona
Valencia
Seville
Zaragoza
Malaga
Bilbao
Santiago de Compostela
Salamanca
Santander
Cáceres
Canary Islands
Balearic Islands
See all the destinations
Search destinations
See all the destinations
What to do
Activities
to suit all tastes
Coasts and beaches
Urban tourism
Nature
Food and wine
Sport and adventure
Shopping
Routes
Plan your trip
Book
your trip to Spain.
Sustainability in Spain
About Spain
Practical information
How to get to Spain
Getting around Spain
Book your experiences
Route planner
Download brochures
The weather
Visa and passport
Sustainability in Spain
Plan an environmentally and socially responsible holiday. Help to care for the planet.
About Spain
Data and interesting facts about the country, its culture, nature, gastronomy and population.
Practical information
Prepare f

In [50]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a travel website \
and creates a short brochure about the destination for prospective travellers, backpackers and adventure lovers. Respond in markdown.\
Include details of destination culture, cities and to do list if you have the information."

In [51]:
def get_brochure_user_prompt(dest, url):
    user_prompt = f"You are looking to travel to: {dest} \n"
    user_prompt += f"Here are the contents of the landing page and other relevant pages; use this information to build a short travel brochure of the destination in markdown.\n"
    user_prompt += get_all_details(url)
    return user_prompt

In [52]:
get_brochure_user_prompt("Spain", "https://www.spain.info/en/")

'You are looking to travel to: Spain \nHere are the contents of the landing page and other relevant pages; use this information to build a short travel brochure of the destination in markdown.\nLanding page:\nWebpage Title: \nSpanish Tourism | Tourist information on Spain | spain.info\nWebpage contents: \nSpain\'s official tourism website\nSearch\nWhere to go\nDiscover\nour destinations\nMadrid\nBarcelona\nValencia\nSeville\nZaragoza\nMalaga\nBilbao\nSantiago de Compostela\nSalamanca\nSantander\nCáceres\nCanary Islands\nBalearic Islands\nSee all the destinations\nSearch destinations\nSee all the destinations\nWhat to do\nActivities\nto suit all tastes\nCoasts and beaches\nUrban tourism\nNature\nFood and wine\nSport and adventure\nShopping\nRoutes\nPlan your trip\nBook\nyour trip to Spain.\nSustainability in Spain\nAbout Spain\nPractical information\nHow to get to Spain\nGetting around Spain\nBook your experiences\nRoute planner\nDownload brochures\nThe weather\nVisa and passport\nSusta

In [53]:
def create_brochure(dest, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(dest, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [54]:
create_brochure("Spain", "https://www.spain.info/en/")

# Discover Spain: Your Ultimate Travel Brochure

Welcome to Spain, a land bursting with vibrant culture, breathtaking landscapes, and culinary delights. Get ready to embark on an adventure through its picturesque cities and stunning natural parks. Whether you're a seasoned traveler, a backpacker, or an adventure lover, Spain has something for everyone!

---

## Cultural Richness

Spain is home to an incredible variety of cultural experiences. From the architectural wonders of Antoni Gaudí in Barcelona to the majestic palaces in Seville, every city tells a unique story. Explore world-renowned museums like the **Prado** and **Reina Sofía** in Madrid, or indulge in the romantic streets of **Salamanca**, a UNESCO World Heritage site known for its stunning sandstone architecture.

### Cities to Visit
- **Madrid**: The buzzing capital renowned for its art, tapas, and lively culture.
- **Barcelona**: A cosmopolitan city with a rich history, beautiful beaches, and landmark structures like the Sagrada Familia.
- **Seville**: Famous for flamenco dancing, rich history, and stunning Moorish architecture.
- **Valencia**: Home to the futuristic **City of Arts and Sciences** and delicious **paella**.
- **Bilbao**: Known for the **Guggenheim Museum** and its breathtaking contrasts between modern architecture and traditional Basque culture.
- **Santiago de Compostela**: The final destination of the Camino de Santiago, famous for its grand cathedral.

---

## Things to Do

With endless activities to choose from, here’s a **to-do list** to help you make the most of your trip:

1. **Culinary Adventures**: Explore Spain's rich gastronomy by trying tapas in Madrid, seafood in Galicia, and world-class wines in La Rioja.
2. **Outdoor Activities**: Enjoy hiking in the stunning national parks, including **Picos de Europa** and **Teide National Park**. Activities include climbing, water sports, and birdwatching.
3. **Art and Festivals**: Experience local festivals like **La Tomatina** in Buñol, **Semana Santa** in Seville, or partake in the vibrant nightlife in Ibiza.
4. **Historical Tours**: Visit ancient ruins in **Cáceres**, and the Roman architecture in **Merida**, or delve into the Jewish heritage in **Toledo**.
5. **Shopping**: Discover local markets, boutique shops, and high-street fashion in cities like Barcelona and Madrid.

---

## Natural Wonders

Spain is not only about bustling cities; it also offers stunning countryside and coastlines. From the beaches of the **Costa del Sol** to the mountains of the **Sierra Nevada**, nature enthusiasts will find countless opportunities to explore:
- **Canary Islands**: Renowned for their volcanic landscapes and ideal climate year-round.
- **Balearic Islands**: Famous for idyllic beaches and crystal-clear waters, offering everything from relaxation to vibrant nightlife.

---

## Plan Your Trip

### Getting Around
Spain’s well-connected transport system makes it easy to travel between cities. High-speed trains (AVE) can whisk you from Madrid to Seville in just a few hours.

### Useful Tips
- Opt for **local buses** or **trams** for city explorations.
- Many cities have **tourist passes** that offer discounts for attractions and public transport.
- Be prepared for numerous outdoor experiences that celebrate Spain's diverse landscapes.

### Seasonal Recommendations
- **Best Time to Visit**: Spring (March to May) and Fall (September to November) when the weather is mild and the tourist crowds are smaller.

---

Experience the warm hospitality, rich history, and scenic beauty that Spain offers. Embark on your adventure today, and let Spain captivate your heart!

---

For more information and to plan your trip, visit our website at [spain.info](https://www.spain.info). Happy travels!